# Advanced Querying Mongo

**⚠️ IMPORTANT: Limit your print to avoid infinite scrolling. Otherwise your
code will be lost between print lines. If working with lists do:**

```python
list(collection.find(query))[:5] #or a reasonably low number
```

Importing libraries and setting up connection

In [22]:
from pymongo import MongoClient
import pandas as pd
client = MongoClient("localhost:27017")

In [5]:
client.list_database_names()

['Crunchbase_db', 'admin', 'config', 'ironhack', 'local']

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [9]:
db = client["Crunchbase_db"]
c = db.get_collection("Crunchbase")

In [15]:
list(c.find({"name":"Babelgum"}, projection = {"name":1, '_id':0}))

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [133]:
#to avoid too long I just limit the answer to name of the commany and number of employees

df=pd.DataFrame(c.find({"number_of_employees": {"$gt": 5000}}, projection = {"name":1, '_id':0, "number_of_employees":1}))
df=df.sort_values(by="number_of_employees", ascending=False).head(20)[:5]
df

,name,number_of_employees
56,Siemens,405000
28,IBM,388000
68,Toyota,320000
9,PayPal,300000
89,Nippon Telegraph and Telephone Corporation,227000


### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [134]:
df=pd.DataFrame(c.find({"founded_year": {"$gte": 2000, '$lte':2005}}, projection = {"name":1, '_id':0, "founded_year":1}))
df=df.sort_values(by="founded_year", ascending=False)
df[:5]

,name,founded_year
0,Wetpaint,2005
1256,EffectiveUI,2005
1281,Halosys,2005
1280,FundingUniverse,2005
3165,Callender Creates,2005


### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [137]:
condition_1 = {"ipo.valuation_amount": {"$gt": 100000000}}
condition_2 = {"founded_year": {"$lt": 2010}}

In [139]:
df=pd.DataFrame(c.find({"$and": [condition_1, condition_2]}, projection= {"name":1, '_id':0, "ipo":1}))
df[:5]

,name,ipo
0,Facebook,"{'valuation_amount': 104000000000.0, 'valuatio..."
1,Twitter,"{'valuation_amount': 18100000000.0, 'valuation..."
2,Yelp,"{'valuation_amount': 1300000000, 'valuation_cu..."
3,LinkedIn,"{'valuation_amount': 9310000000.0, 'valuation_..."
4,Amazon,"{'valuation_amount': 100000000000.0, 'valuatio..."


### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [59]:
condition_1 = {"number_of_employees": {"$lt": 1000}}
condition_2 = {"founded_year": {"$lt": 2005}}

In [140]:
df=pd.DataFrame(c.find({"$and": [condition_1, condition_2]}, projection= {"name":1, '_id':0, "number_of_employees":1,'founded_year':1}))
df=df.sort_values(by="number_of_employees", ascending=False).head(10)
df[:5]

,name,number_of_employees,founded_year
34,Groupon,10000.0,2008
24,Baidu,6000.0,1999
0,Facebook,5299.0,2004
3,LinkedIn,4800.0,2003
15,Salesforce,3500.0,1999


### 6. All the companies that don't include the `partners` field.

In [141]:
df=pd.DataFrame(c.find({"partners":[]}, projection = {"name":1, '_id':0, "partners":1}))
df[:5]

,name,partners
0,Wetpaint,[]
1,AdventNet,[]
2,Zoho,[]
3,Digg,[]
4,Facebook,[]


### 7. All the companies that have a null type of value on the `category_code` field.

In [142]:
df=pd.DataFrame(c.find({"category_code": {"$type": 10}}, projection = {"name":1, '_id':0, "category_code":1}))
df[:5]

,name,category_code
0,Collective,None
1,Snimmer,None
2,KoolIM,None
3,Level9 Media,None
4,VidKing,None


### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [143]:
df=pd.DataFrame(c.find({"number_of_employees": {"$gte": 100, '$lte':1000}}, projection = {"name":1, '_id':0, "number_of_employees":1}))
df[:5]

,name,number_of_employees
0,AdventNet,600
1,AddThis,120
2,OpenX,305
3,LifeLock,644
4,Jajah,110


### 9. Order all the companies by their IPO price in a descending order.

In [144]:
result =list(c.find({}, projection={"name": 1, "_id": 0, "ipo.valuation_amount": 1}).sort("ipo.valuation_amount",-1))
result[:5]


[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000.0}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000.0}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000.0}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000.0}},
 {'name': 'Groupon', 'ipo': {'valuation_amount': 12800000000.0}}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [145]:
result =list(c.find({}, projection={"name": 1, "_id": 0, "number_of_employees": 1}).sort("number_of_employees",-1).limit(10))
result[:5]

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [146]:
filters = {"founded_month": {"$gt": 6}}
projection = {'name': 1, "_id": 0, "founded_month": 1}
result = list(c.find(filters, projection).limit(1000))
result[:5]

[{'name': 'Wetpaint', 'founded_month': 10},
 {'name': 'Zoho', 'founded_month': 9},
 {'name': 'Digg', 'founded_month': 10},
 {'name': 'Omnidrive', 'founded_month': 11},
 {'name': 'eBay', 'founded_month': 9}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.000

In [147]:
condition_1 = {"founded_year": {"$lt": 2000}}
condition_2 = {"ipo.valuation_amount": {"$gt": 10000000}}
filters = {"$and": [condition_1, condition_2]}
projection = {'name': 1, "_id": 0, "founded_year": 1 ,"valuation_amount": 1}
result = list(c.find(filters, projection))
result[:5]

[{'name': 'Amazon', 'founded_year': 1994},
 {'name': 'Nielsen', 'founded_year': 1922},
 {'name': 'OpenTable', 'founded_year': 1998},
 {'name': 'Shutterfly', 'founded_year': 1999},
 {'name': 'Salesforce', 'founded_year': 1999}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [148]:
filters = {"acquisition.acquired_year": {"$gt": 2010}}
projection = {'name': 1, "_id": 0, "acquisition": 1}
result = list(c.find(filters, projection).sort("acquisition.price_amount",-1))
result[:5]

[{'name': 'T-Mobile',
  'acquisition': {'price_amount': 39000000000.0,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/03/20/in-the-race-for-more-spectrum-att-is-acquiring-t-mobile-for-39-billion/',
   'source_description': 'In The Race For More Spectrum, AT&T Is Acquiring T-Mobile For $39 Billion',
   'acquired_year': 2011,
   'acquired_month': 3,
   'acquired_day': 20,
   'acquiring_company': {'name': 'AT&T', 'permalink': 'at-t'}}},
 {'name': 'Goodrich Corporation',
  'acquisition': {'price_amount': 18400000000.0,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://www.masshightech.com/stories/2011/09/19/daily37-UTC-shells-out-184-billion-for-Goodrich.html',
   'source_description': 'UTC shells out $18.4 billion for Goodrich',
   'acquired_year': 2011,
   'acquired_month': 9,
   'acquired_day': 22,
   'acquiring_company': {'name': 'United Technologies',
    'permalink': 'united-technologies'}}},
 {'name':

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [149]:
filters = {"founded_year": {"$exists": True,"$ne": None}}
projection = {'name': 1, "_id": 0, "founded_year": 1}
result = list(c.find(filters, projection).sort("founded_year",-1).sort("founded_year"))
result[:5]

[{'name': 'Alstrasoft', 'founded_year': 1800},
 {'name': 'SmallWorlds', 'founded_year': 1800},
 {'name': 'US Army', 'founded_year': 1800},
 {'name': 'DuPont', 'founded_year': 1802},
 {'name': 'McKesson', 'founded_year': 1833}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [150]:
filters = {"founded_day": {"$lte": 7}}
projection = {'name': 1, "_id": 0, "founded_day": 1, "acquisition.price_amount" : 1}
result = list(c.find(filters, projection).sort("acquisition.price_amount",-1).limit(10))
result[:5]

[{'name': 'Netscape',
  'founded_day': 4,
  'acquisition': {'price_amount': 4200000000.0}},
 {'name': 'PayPal',
  'founded_day': 1,
  'acquisition': {'price_amount': 1500000000}},
 {'name': 'Zappos',
  'founded_day': 1,
  'acquisition': {'price_amount': 1200000000}},
 {'name': 'Alibaba',
  'founded_day': 1,
  'acquisition': {'price_amount': 1000000000}},
 {'name': 'Postini',
  'founded_day': 2,
  'acquisition': {'price_amount': 625000000}}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [153]:
condition_1 = {"category_code": 'web'}
condition_2 = {"number_of_employees": {"$gt": 4000}}

filters = {"$and": [condition_1, condition_2]}
projection = {'name': 1, "_id": 0, "category_code": 1, "number_of_employees" : 1}

result = list(c.find(filters, projection).sort("number_of_employees",-1))
result[:5]

[{'name': 'Experian', 'category_code': 'web', 'number_of_employees': 15500},
 {'name': 'eBay', 'category_code': 'web', 'number_of_employees': 15000},
 {'name': 'Yahoo!', 'category_code': 'web', 'number_of_employees': 13600},
 {'name': 'Rakuten', 'category_code': 'web', 'number_of_employees': 10000},
 {'name': 'Los Angeles Times Media Group',
  'category_code': 'web',
  'number_of_employees': 10000}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [155]:
condition_1 = {"acquisition.price_currency_code": 'EUR'}
condition_2 = {"acquisition.price_amount": {"$gt": 10000000}}

filters = {"$and": [condition_1, condition_2]}
projection = {'name': 1, "_id": 0, "acquisition.price_amount": 1, "acquisition.price_currency_code" : 1}

result = list(c.find(filters, projection).sort("number_of_employees",-1))
result[:5]

[{'name': 'Tuenti Technologies',
  'acquisition': {'price_amount': 70000000, 'price_currency_code': 'EUR'}},
 {'name': 'Webedia',
  'acquisition': {'price_amount': 70000000, 'price_currency_code': 'EUR'}},
 {'name': 'ZYB',
  'acquisition': {'price_amount': 31500000, 'price_currency_code': 'EUR'}},
 {'name': 'Apertio',
  'acquisition': {'price_amount': 140000000, 'price_currency_code': 'EUR'}},
 {'name': 'Greenfield Online',
  'acquisition': {'price_amount': 40000000, 'price_currency_code': 'EUR'}}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [157]:
filters = {"acquisition.acquired_month": {"$lte": 3}}
projection = {'name': 1, "_id": 0, "acquisition": 1}

result = list(c.find(filters, projection).limit(10))
result[:5]

[{'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'name': 'NetRatings',
  'acquisition': {'price_amount': 327000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://login.vnuemedia.com/hr/login/login_subscribe.jsp?id=0oqDem1gYIfIclz9i2%2Ffqj5NxCp2AC5DPbVnyT2da8GyV2mXjasabE128n69OrmcAh52%2FGE3pSG%2F%0AEKRYD9vh9EhrJrxukmUzh532fSMTZXL42gwPB80UWVtF1NwJ5UZSM%2BCkLU1mpYBoHFgiH%2Fi0f6Ax%0A9yMIVxt47t%2BHamhEQ0nkOEK24L',
   'source_description': 'Nielsen buys rest of NetRatings',
   'acquired_year': 2007,
   'acquired_month': 2,
   

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code

**⚠️ Did you do this?**

```python
list(collection.find(query))[:5] #or a reasonably low number
```